# 4.3 오버피팅과 정규화 (Overfitting and Regularization)

머신러닝 모델
과적합(Overfitting)

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import transforms, datasets

In [2]:
torch.manual_seed(42)
USE_CUDA = torch.cuda.is_available()
DEVICE = torch.device("cuda" if USE_CUDA else "cpu")

In [3]:
EPOCHS = 50
BATCH_SIZE = 64

## 데이터셋에 노이즈 추가하기

![original.png](./assets/original.png)

![horizontalflip.png](./assets/horizontalflip.png)

In [4]:
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('./.data',
                   train=True,
                   download=True,
                   transform=transforms.Compose([
                       transforms.RandomHorizontalFlip(),
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=BATCH_SIZE, shuffle=True)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('./.data',
                   train=False, 
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=BATCH_SIZE, shuffle=True)

## 뉴럴넷으로 Fashion MNIST 학습하기

입력 `x` 는 `[배치크기, 색, 높이, 넓이]`로 이루어져 있습니다.
`x.size()`를 해보면 `[64, 1, 28, 28]`이라고 표시되는 것을 보실 수 있습니다.
Fashion MNIST에서 이미지의 크기는 28 x 28, 색은 흑백으로 1 가지 입니다.
그러므로 입력 x의 총 특성값 갯수는 28 x 28 x 1, 즉 784개 입니다.

우리가 사용할 모델은 3개의 레이어를 가진 뉴럴네트워크 입니다. 

In [5]:
class Net(nn.Module):
    def __init__(self, dropout_p=0.2):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(784, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 10)
        self.dropout_p = dropout_p

    def forward(self, x):
        x = x.view(-1, 784)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training,
                      p=self.dropout_p)
        x = F.relu(self.fc2(x))
        x = F.dropout(x, training=self.training,
                      p=self.dropout_p)
        x = self.fc3(x)
        return x

## 모델 준비하기 

`to()` 함수는 모델의 파라미터들을 지정한 곳으로 보내는 역할을 합니다.
일반적으로 CPU 1개만 사용할 경우 필요는 없지만,
GPU를 사용하고자 하는 경우 `to("cuda")`로 지정하여 GPU로 보내야 합니다.
지정하지 않을 경우 계속 CPU에 남아 있게 되며 빠른 훈련의 이점을 누리실 수 없습니다.

최적화 알고리즘으로 파이토치에 내장되어 있는 `optim.SGD`를 사용하겠습니다.

In [6]:
model        = Net(dropout_p=0.2).to(DEVICE)
optimizer    = optim.SGD(model.parameters(), lr=0.01)

## 훈련하기

In [7]:
def train(model, train_loader, optimizer):
    model.train()
    for data, target in enumerate(train_loader):
        data, target = data.to(DEVICE), target.to(DEVICE)
        optimizer.zero_grad()
        output = model(data)
        loss = F.cross_entropy(output, target)
        loss.backward()
        optimizer.step()

## 테스트하기

아무리 훈련이 잘 되었다고 해도 실제 데이터를 만났을때 성능이 낮다면 쓸모 없는 모델일 것입니다.
우리가 진정 원하는 것은 훈련 데이터에 최적화한 모델이 아니라 모든 데이터에서 높은 성능을 보이는 모델이기 때문입니다.
세상에 존재하는 모든 데이터에 최적화 하는 것을 "일반화"라고 부르고
모델이 얼마나 실제 데이터에 적응하는지를 수치로 나타낸 것을 "일반화 오류"(Generalization Error) 라고 합니다. 

우리가 만든 모델이 얼마나 일반화를 잘 하는지 알아보기 위해,
그리고 언제 훈련을 멈추어야 할지 알기 위해
매 이포크가 끝날때 마다 테스트셋으로 모델의 성능을 측정해보겠습니다.

In [8]:
def test(model, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(DEVICE), target.to(DEVICE)
            output = model(data)
            test_loss += F.cross_entropy(output, target,
                                         size_average=False).item()
            
            # 맞춘 갯수 계산
            pred = output.max(1, keepdim=True)[1]
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    test_accuracy = 100. * correct / len(test_loader.dataset)
    return test_loss, test_accuracy

## 코드 돌려보기

자, 이제 모든 준비가 끝났습니다. 코드를 돌려서 실제로 훈련이 되는지 확인해봅시다!

In [9]:
for epoch in range(1, EPOCHS + 1):
    train(model, train_loader, optimizer)
    test_loss, test_accuracy = test(model, test_loader)
    
    print('[{}] Test Loss: {:.4f}, Accuracy: {:.2f}%'.format(
          epoch, test_loss, test_accuracy))

[1] Test Loss: 0.5499, Accuracy: 82.68%
[2] Test Loss: 0.4284, Accuracy: 86.36%
[3] Test Loss: 0.3552, Accuracy: 89.13%
[4] Test Loss: 0.3043, Accuracy: 90.44%
[5] Test Loss: 0.2602, Accuracy: 92.07%
[6] Test Loss: 0.2321, Accuracy: 92.82%
[7] Test Loss: 0.2132, Accuracy: 93.41%
[8] Test Loss: 0.1986, Accuracy: 93.85%
[9] Test Loss: 0.1824, Accuracy: 94.46%
[10] Test Loss: 0.1756, Accuracy: 94.56%
[11] Test Loss: 0.1665, Accuracy: 94.87%
[12] Test Loss: 0.1555, Accuracy: 95.30%
[13] Test Loss: 0.1524, Accuracy: 95.39%
[14] Test Loss: 0.1451, Accuracy: 95.58%
[15] Test Loss: 0.1399, Accuracy: 95.78%
[16] Test Loss: 0.1370, Accuracy: 95.82%
[17] Test Loss: 0.1339, Accuracy: 95.93%
[18] Test Loss: 0.1289, Accuracy: 96.08%
[19] Test Loss: 0.1239, Accuracy: 96.17%
[20] Test Loss: 0.1194, Accuracy: 96.27%
[21] Test Loss: 0.1153, Accuracy: 96.47%
[22] Test Loss: 0.1183, Accuracy: 96.20%
[23] Test Loss: 0.1140, Accuracy: 96.49%
[24] Test Loss: 0.1101, Accuracy: 96.60%
[25] Test Loss: 0.1088, A